<a href="https://colab.research.google.com/github/joshmcadams/nanoGPT/blob/main/nanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nanoGPT

nanoGPT is a wonderful exploration into building a GPT model from scratch. We'll start with an empty notebook and quickly build and train a model to generate Shakespeare-like text.

This notebook is based off of a code and videos releasted by Andrej Karpathy. Please be sure to check out his work!

  * [Reference Video Lecture](https://www.youtube.com/watch?v=kCc8FmEb1nY)
  * [Reference Git Repository](https://github.com/karpathy/ng-video-lecture)

# Data Exploration and Preparation

## Acquiring the data

We'll first pull the data down into this lab. To get the data the `wget` command will be used in the shell.

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-05-12 05:05:57--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2024-05-12 05:05:57 (31.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



Next we'll load the data into Python and take a look at a sample.

First, we simply open the file and store the entire contents of the file in the variable `raw_training_data`. We get the length of the data and print that out so that we can see how much data we are dealing with.

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    raw_training_data = f.read()

training_data_size = len(raw_training_data)
print(training_data_size)

1115394


Next we sample the data just to get an idea of what we are dealing with. The sample comes from a random locaiton in the data. Run the code block below a few times to see some different data samples.

In [3]:
import random

SAMPLE_SIZE = 500
start = random.randrange(0, training_data_size - SAMPLE_SIZE)
print(raw_training_data[start:start+SAMPLE_SIZE])

ed
putting-on; methinks strangely, for he hath not used it before.

DUKE VINCENTIO:
Pray you, let's hear.

Provost:

DUKE VINCENTIO:
What is that Barnardine who is to be executed in the
afternoon?

Provost:
A Bohemian born, but here nursed un and bred; one
that is a prisoner nine years old.

DUKE VINCENTIO:
How came it that the absent duke had not either
delivered him to his liberty or executed him? I
have heard it was ever his manner to do so.

Provost:
His friends still wrought reprieves for h


## Data Analysis

We've now acquired the data and even poked around a bit to see what the data looks like. However, we need to dig deeper and do some more detailed analysis on the data.

One important fact to know about data that will be fed into a generative model is how many different characters are we working with. This is actually really easy in Python and can be accomplished using the `set` function.

In [4]:
tokens = sorted(list(set(raw_training_data)))
token_count = len(tokens)

print(f'There are {token_count} unique characters in the {training_data_size}' +
      f'of training data. The characters are: {"".join(tokens)}')

There are 65 unique characters in the 1115394of training data. The characters are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


## Encoding

We now know how many different unique tokens exist in our training data. In our case, these tokens will be individual characters, but this a choice that we have made. We could have alternatively chosen pairs, triplets, or some other combination of characters to be a token.

Regardless of the data that comprises our tokens, the token itself isn't what we feed to the model. Instead, we need to convert the token into a numeric representation since models are performing numeric calculations internally.

For our character-based tokens we could just use the ASCII value of the character using the `ord` function. However, you might eventually want to expermiment with sequences of characters as tokens, and in this case you'll need a more complex token-to-number mapping, so lets just create a token-to-number mapping scheme now.

Since we ordered our unique tokens into the `tokens` list, we can just use the index of the token in the list as the encoding, which is what we do.

Note that this does come with some trade-offs though. The position of any given token in the list of tokens is relative to how many and what were the unique tokens in the training data. If we use different trianing data, the tokens might map to different indexes so we need to be careful to preserve our tokens across training data set if we are using mulitple different pieces of training data.

Anyway, let's write a token-to-number `encode` function and a number-to-token `decode` function.

First, create a mapping of tokens to numbers.

In [5]:
token_to_number = {t: i for i, t in enumerate(tokens)}
token_to_number

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

And then a mapping from numbers to tokens.

In [6]:
number_to_token = {i: t for i, t in enumerate(tokens)}
number_to_token

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'A',
 14: 'B',
 15: 'C',
 16: 'D',
 17: 'E',
 18: 'F',
 19: 'G',
 20: 'H',
 21: 'I',
 22: 'J',
 23: 'K',
 24: 'L',
 25: 'M',
 26: 'N',
 27: 'O',
 28: 'P',
 29: 'Q',
 30: 'R',
 31: 'S',
 32: 'T',
 33: 'U',
 34: 'V',
 35: 'W',
 36: 'X',
 37: 'Y',
 38: 'Z',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}

And now we can write our encoder.

In [7]:
def encode(tokens: str) -> list[int]:
  return [token_to_number[t] for t in tokens]

encode('apple')

[39, 54, 54, 50, 43]

And the decoder.

In [8]:
def decode(numbers: list[int]) -> str:
  return ''.join(number_to_token[n] for n in numbers)

decode([16, 17, 18])

'DEF'

We now have a tokenizer and a large amount of text. Let's tokenize all the text!

[Video Reference Point](https://youtu.be/kCc8FmEb1nY?t=778)

In [9]:
import torch
data = torch.tensor(encode(raw_training_data), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


Now we need to split training data and testing data.

In [10]:
training_data_size = int(0.9*len(data))
training_data = data[:training_data_size]
validation_data = data[training_data_size:]
print(len(training_data), len(validation_data))

1003854 111540


## Time Dimension

Next we work on training in blocks.
In this case we train in blocks of 8 characters, but we always need the "next" character, so we work with block sizes of bock+1.

[Video Reference Point](https://youtu.be/kCc8FmEb1nY?t=913)

In [11]:
block_size = 8
training_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = training_data[:block_size]
y = training_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"in: {context}, out: {target}")

in: tensor([18]), out: 47
in: tensor([18, 47]), out: 56
in: tensor([18, 47, 56]), out: 57
in: tensor([18, 47, 56, 57]), out: 58
in: tensor([18, 47, 56, 57, 58]), out: 1
in: tensor([18, 47, 56, 57, 58,  1]), out: 15
in: tensor([18, 47, 56, 57, 58,  1, 15]), out: 47
in: tensor([18, 47, 56, 57, 58,  1, 15, 47]), out: 58


## Batch Dimension

[Video Reference Point](https://youtu.be/kCc8FmEb1nY?t=1129)

In [13]:
torch.manual_seed(1337)

batch_size = 4
block_size = 8

def get_batch(split):
  data = training_data if split == 'train' else validation_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)
print('----------------------------------')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f'input {context.tolist()}, target {target}')

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----------------------------------
input [24], target 43
input [24, 43], target 58
input [24, 43, 58], target 5
input [24, 43, 58, 5], target 57
input [24, 43, 58, 5, 57], target 1
input [24, 43, 58, 5, 57, 1], target 46
input [24, 43, 58, 5, 57, 1, 46], target 43
input [24, 43, 58, 5, 57, 1, 46, 43], target 39
input [44], target 53
input [44, 53], target 56
input [44, 53, 56], target 1
input [44, 53, 56, 1], target 58
input [44, 53, 56, 1, 58], target 46
input [44, 53, 56, 1, 58, 46], target 39
input [44, 53, 56, 1, 58, 46, 39], target 58
input [44, 53, 56, 1, 58, 46, 39, 58], target 1
input [52], tar

# Bigram Model

## Generation

[Video Reference Point](https://youtu.be/kCc8FmEb1nY?t=1348)

[Makemore Series covering the BiGram language model](https://www.youtube.com/watch?v=PaCmpygFfXo)

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets):
    logits = self.token_embedding_table(idx)

    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)
    loss = F.cross_entropy(logits, targets)

    return logits, loss

m = BigramLanguageModel(token_count)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [15]:
import numpy as np

print(-np.log(1/65.0))

4.174387269895637


[Video Reference Point](https://youtu.be/kCc8FmEb1nY?t=1753)

In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :] # (B, C)
      probs = F.softmax(logits, dim=-1) # (B, C)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

m = BigramLanguageModel(token_count)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

z = torch.zeros((1,1), dtype=torch.long)
print(z)
generated = m.generate(z, max_new_tokens=100)
print(generated)
generated_list = generated[0].tolist()
print(generated_list)
print(decode(generated_list))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
tensor([[0]])
tensor([[ 0, 31, 56, 12, 55, 28,  7, 29, 35, 49, 58, 36, 53, 24,  4, 48, 24, 16,
         22, 45, 27, 24, 34, 64,  5, 30, 21, 53, 16, 55, 20, 42, 46, 57, 34,  4,
         60, 24, 24, 62, 39, 58, 48, 57, 41, 25, 54, 61, 24, 17, 30, 31, 28, 63,
         39, 53,  8, 55, 44, 64, 57,  3, 37, 57,  3, 64, 18,  7, 61,  6, 11, 43,
         17, 49, 64, 62, 48, 45, 15, 23, 18, 15, 46, 57,  2, 47, 35, 35,  8, 27,
         40, 64, 16, 52, 62, 13,  1, 25, 57,  3,  9]])
[0, 31, 56, 12, 55, 28, 7, 29, 35, 49, 58, 36, 53, 24, 4, 48, 24, 16, 22, 45, 27, 24, 34, 64, 5, 30, 21, 53, 16, 55, 20, 42, 46, 57, 34, 4, 60, 24, 24, 62, 39, 58, 48, 57, 41, 25, 54, 61, 24, 17, 30, 31, 28, 63, 39, 53, 8, 55, 44, 64, 57, 3, 37, 57, 3, 64, 18, 7, 61, 6, 11, 43, 17, 49, 64, 62, 48, 45, 15, 23, 18, 15, 46, 57, 2, 47, 35, 35, 8, 27, 40, 64, 16, 52, 62, 13, 1, 25, 57, 3, 9]

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;e

## Training

[Video Reference Point](https://youtu.be/kCc8FmEb1nY?t=2092)

In [17]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [23]:
batch_size = 32

for steps in range(10000):
  xb, yb = get_batch('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.559082508087158


In [24]:
z = torch.zeros((1,1), dtype=torch.long)
print(z)
generated = m.generate(z, max_new_tokens=100)
print(generated)
generated_list = generated[0].tolist()
print(generated_list)
print(decode(generated_list))

tensor([[0]])
tensor([[ 0, 13, 52, 41, 53,  1, 57,  1, 39, 54, 56, 45, 56,  1, 39, 50, 57,  1,
         53, 56, 43, 56, 60, 43,  6,  1, 63, 53,  1, 47, 42, 47, 50, 39, 41, 39,
         47, 39,  1, 44,  1, 58, 50, 50, 47, 52, 45, 47, 58, 56, 43, 63,  1, 50,
          1, 53, 59, 43, 60, 43, 56, 58,  1, 15, 33, 30, 53, 56, 42,  1, 40, 43,
         39,  1, 39, 56, 53, 59, 50, 50, 39, 47, 53, 61, 53, 52, 45, 43,  6,  1,
         57,  8,  0, 25, 63,  1, 63,  1, 48, 53, 51]])
[0, 13, 52, 41, 53, 1, 57, 1, 39, 54, 56, 45, 56, 1, 39, 50, 57, 1, 53, 56, 43, 56, 60, 43, 6, 1, 63, 53, 1, 47, 42, 47, 50, 39, 41, 39, 47, 39, 1, 44, 1, 58, 50, 50, 47, 52, 45, 47, 58, 56, 43, 63, 1, 50, 1, 53, 59, 43, 60, 43, 56, 58, 1, 15, 33, 30, 53, 56, 42, 1, 40, 43, 39, 1, 39, 56, 53, 59, 50, 50, 39, 47, 53, 61, 53, 52, 45, 43, 6, 1, 57, 8, 0, 25, 63, 1, 63, 1, 48, 53, 51]

Anco s aprgr als orerve, yo idilacaia f tllingitrey l ouevert CURord bea aroullaiowonge, s.
My y jom


# When the code is converted to a script


[Video Checkpoint](https://youtu.be/kCc8FmEb1nY?t=2257)

In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

torch.manual_seed(1337)

with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)


stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"iter {iter}; train loss {losses['train']:.4f}; val loss {losses['val']:.4f}")

  xb, yb = get_batch('train')

  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


iter 0 losses {'train': tensor(4.7305), 'val': tensor(4.7241)}
iter 300 losses {'train': tensor(2.8101), 'val': tensor(2.8243)}
iter 600 losses {'train': tensor(2.5400), 'val': tensor(2.5652)}
iter 900 losses {'train': tensor(2.4896), 'val': tensor(2.5060)}
iter 1200 losses {'train': tensor(2.4830), 'val': tensor(2.5012)}
iter 1500 losses {'train': tensor(2.4631), 'val': tensor(2.4905)}
iter 1800 losses {'train': tensor(2.4655), 'val': tensor(2.4926)}
iter 2100 losses {'train': tensor(2.4670), 'val': tensor(2.4840)}
iter 2400 losses {'train': tensor(2.4612), 'val': tensor(2.4879)}
iter 2700 losses {'train': tensor(2.4717), 'val': tensor(2.4915)}

od nos CAy go ghanoray t, co haringoudrou clethe k, leve fr werar,
Is fa!


Thilemel cia h hmboomyorarifrcitheviPO, tle dst f qur'dig t cof boddo y t o ar pileas h mo wierl t,
S:
STENENEat I athe thounomy tinrent distesisanimald 3I'leliento ald, avavis nofrisist me Busarend un'soto iat s k,
SBRI he the f wendleindd t acoe ts ansur thy ppr h.



# Math Trick Self-Attention

In [41]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [44]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1]
    xbow[b,t] = torch.mean(xprev, 0)

x[0], xbow[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [45]:
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('----------------')
print('b=')
print(b)
print('----------------')
print('c=')
print(c)
print('----------------')

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
----------------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----------------
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
----------------


In [49]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [50]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('----------------')
print('b=')
print(b)
print('----------------')
print('c=')
print(c)
print('----------------')

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
----------------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----------------
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])
----------------


In [51]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('----------------')
print('b=')
print(b)
print('----------------')
print('c=')
print(c)
print('----------------')

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----------------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----------------
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
----------------


In [52]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [57]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C)
torch.allclose(xbow[0], xbow2[0])

True

In [56]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [61]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow[0], xbow3[0])

True

# Adding Self-Attention

[Video Checkpoint](https://youtu.be/kCc8FmEb1nY?t=3510)

In [62]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32

torch.manual_seed(1337)

with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)


stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, idx, targets=None):
    tok_emb = self.token_embedding_table(idx)
    logits = self.lm_head(tok_emb)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

model = BigramLanguageModel()
m = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"iter {iter}; train loss {losses['train']:.4f}; val loss {losses['val']:.4f}")

  xb, yb = get_batch('train')

  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


iter 0; train loss 4.3886; val loss 4.3734
iter 300; train loss 2.5268; val loss 2.5406
iter 600; train loss 2.5004; val loss 2.5331
iter 900; train loss 2.4910; val loss 2.5101
iter 1200; train loss 2.4985; val loss 2.5155
iter 1500; train loss 2.4819; val loss 2.5046
iter 1800; train loss 2.4864; val loss 2.5177
iter 2100; train loss 2.4870; val loss 2.5024
iter 2400; train loss 2.4892; val loss 2.5159
iter 2700; train loss 2.5014; val loss 2.5142

balyorerasicallcod nos CAneno ghalousord, co hakingoudre h wethe kes une fr werar,
Is falo he.
IUCis cou. ppr:


IOKarifrcithevico, tld dst f Couced thin f boddo y t o ar pileas h mavimerl t,
S:
STo'd cackepathe thounomertinrent distesisanimald 3 m ensento ald, avaviconofrisist menghallpr, un'soto iat s kn, use hepute f fendleindd t acoe ts ansur thy ppr h.


Y:
KIIsqu pr oded char whave ousenlll owhofrd miner t ooon'stoume wh domo! fifoveghind hiarnge s.
MENaswimy ojomes ard'd; 'To tee abw beme
